In [84]:
import gensim
from collections import namedtuple
from collections import OrderedDict
import multiprocessing
import numpy as np
import pandas as pd
import json
import logging
import random
import pymongo
from pymongo import MongoClient
from random import sample
import statsmodels.api as sm
from gensim.models import Doc2Vec
import gensim.models.doc2vec
from gensim.models import Word2Vec
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models.doc2vec import TaggedDocument
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics.pairwise import cosine_similarity
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec

# for timing
from contextlib import contextmanager
from timeit import default_timer
import time

from random import shuffle
import datetime

In [88]:
client = MongoClient()
db = client['tweets_db']
cursor = db.tweets.find()
count = 2
df = pd.DataFrame(columns=['tweet_id', 'text', 'timestamp'])
for tweet in cursor:
    count -= 1
    try:
        if 'text' in tweet:
            data = pd.read_json(tweet)
#             df.append(data)
            print data
#             df.append(tweet)
#             print tweet['id'] 
#             print tweet['created_at'] 
#             print tweet['text']
#             hashtags = []
#             for hashtag in tweet['entities']['hashtags']:
#             	hashtags.append(hashtag['text'])
#             print hashtags

    except:
        # read in a line is not in JSON format (sometimes error occured)
        continue
    if count <= 0:
        break

In [49]:
vectorizer = CountVectorizer(stop_words='english')
vectors = vectorizer.fit_transform(data).toarray()
words = vectorizer.get_feature_names()